In [ ]:
import cv2
import numpy as np
import socket
import struct

# Define the host and port for the server
HOST = '127.0.0.1'
PORT = 9999

# Create a socket and bind it to the host and port
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(1)
    print(f"Waiting for a connection on {HOST}:{PORT}")

    # Accept a connection from a client
    conn, addr = s.accept()
    print(f"Connected to {addr}")

    # Loop to receive images from the client
    while True:
        # Receive the size of the image data
        size_data = conn.recv(struct.calcsize("L"))

        # Check if the size data is empty, which indicates that the client has closed the connection
        if not size_data:
            print("Connection closed by client")
            break

        # Unpack the size data and receive the image data
        size = struct.unpack("L", size_data)[0]
        data = b""
        while len(data) < size:
            packet = conn.recv(size - len(data))
            if not packet:
                break
            data += packet

        # Check if the image data is empty, which indicates that the client has closed the connection
        if not data:
            print("Connection closed by client")
            break

        # Decode the image data and display the image
        nparr = np.frombuffer(data, np.uint8)
        try:
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            cv2.imshow('Received Image', img)
            cv2.waitKey(1)
        except Exception as e:
            print("Error decoding image:", e)

    # Release resources and close the connection
    cv2.destroyAllWindows()
    conn.close()
    print("Connection closed")

Waiting for a connection on localhost:9999
